Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='green'><b>Полезные (и просто интересные) материалы:</b></font> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 

Если понравилась работа с текстами, то можешь посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .
</font>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Радует, что ноутбук хорошо структурирован. Приятно проверять такие работы.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

# анализ отзывов для «Викишоп» с BERT

# imports

In [1]:
#!pip install transformers
#!pip install catboost
!pip install detoxify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 6.9 MB/s eta 0:00:0000:01m00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-38
      creating build/lib.macosx-11.1-arm64-cpython-38/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.macosx-11.1-arm64-cpython-38/tokenizers
      creating build/lib.macosx-11.1-arm64-cpython-38/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.maco

In [3]:
import pandas as pd 
import numpy as np 

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import RandomizedSearchCV

import torch
import transformers
from tqdm import notebook
from torch import tensor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re
from sklearn.utils import shuffle

from catboost import Pool, CatBoostClassifier

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [4]:
DATA_PATH = 'toxic_comments.csv'
#DATA_PATH = '/datasets/toxic_comments.csv'

In [5]:
# напишем функцию для очистки и лемматизации текста с маркированными частями речи
def clear_text(text):
  #очистка текста 
  return ' '.join(re.sub(r'[^a-zA-Z\' ]', ' ', text).split())

In [6]:
#проверка на GPU, если есть, то ускорит BERT
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


# first look 

In [8]:
df = pd.read_csv(DATA_PATH)
initial_df = df.copy()

In [9]:
df.sample(10)

,Unnamed: 0,text,toxic
69450,69518,"""\n\nWhat exactly is it about my """"editing sty...",0
35974,36016,"Welcome!\n\nHello, , and welcome to Wikipedia!...",0
59880,59946,i tried but the chart kept messing up when i c...,0
34153,34193,""" Let's discuss it """,0
53013,53070,"""\n Yep. This is one of several cases where t...",0
129751,129884,"Category:People from Westfield, Wisconsin\n\n:...",0
104647,104744,Happy to be of some help. It's not hard to pro...,0
23678,23698,|decline=Fucking bellend —,1
135702,135840,Simon Mills is pretty gay. It regularly takes ...,1
58396,58462,Re: WWII France articles mess: occupation edit...,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


# data preprocessing

In [11]:
df = df.drop(['Unnamed: 0'], axis = 1) 

In [12]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [13]:
df.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены корреткно. Радует, что баланс классов был изучен.
</div>

таргет сильно несбалансирован, лучше сделать downsampling, так модель луше выхватит целевой признак, заодно и сэко

In [14]:
#df = pd.concat([df.query('toxic == 1'), df.query('toxic != 1').sample(20000, replace = False)])
#df = shuffle(df)

#df = df.sample(frac=0.1, replace=False)

In [15]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [16]:
df['text'] = df['text'].apply(lambda x: clear_text(x))

In [17]:
df['text'] = df['text'].str.lower()

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Изменять баланс классов можно только в треине. В тесте и в валидации он должен быть такой же, как и в исходной выборке. Иначе, результаты получаются некорректными.
</div>

# catboost baseline

In [18]:
df

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww he matches this background colour i'm se...,0
2,hey man i'm really not trying to edit war it's...,0
3,more i can't make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159287,and for the second time of asking when your vi...,0
159288,you should be ashamed of yourself that is a ho...,0
159289,spitzer umm theres no actual article for prost...,0
159290,and it looks like it was actually you who put ...,0


In [19]:
# разделим выборки на обучающую, валидационную и тестовую в пропорции 60%-20%-20%
target = df['toxic']
features = df[['text']]

#указываем stratify=target, чтобы сохранить соотношение классов при разделении
#сначала разделим на обучающую и тестовую в соотношении 60%-40%
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.4, random_state=42, stratify=target) 

#разделим тестовую выборку пополам на тестовую и валидационную
x_valid, x_test, y_valid, y_test = train_test_split(
    x_test, y_test, test_size=0.50, random_state=42)

In [20]:
y_valid

115911    0
56948     0
53584     0
131745    0
81284     0
         ..
148744    0
100014    0
72921     0
117319    0
16215     1
Name: toxic, Length: 31858, dtype: int64

In [23]:
model = CatBoostClassifier(text_features = ['text'], 
                           random_seed = 42, 
                           eval_metric= 'F1',
                           #task_type= 'GPU', 
                           verbose = False)
model.fit(x_train, y_train)

In [24]:
predicted_valid = model.predict(x_valid)
print("F1_score:", f1_score(y_valid, predicted_valid))

F1_score: 0.7678540698674926


получается, что катбуст из коробки уже показывает нужный результат, хорошая штука. 

# BERT

In [25]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
                                                    transformers.DistilBertTokenizer,
                                                    'distilbert-base-uncased')

# model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, 
#                                                     transformers.BertTokenizer, 
#                                                     'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
tokenized= df['text'].apply(lambda x: \
                             tokenizer.encode(x, add_special_tokens=True, 
                                              truncation=True, max_length=512))

In [27]:
tokenized[x_train.index]

155914    [101, 2265, 2033, 1037, 2512, 13433, 2615, 312...
155946    [101, 3374, 4087, 3291, 2007, 1996, 2682, 2930...
91674     [101, 8945, 3022, 1055, 3431, 1999, 20445, 243...
48586     [101, 2417, 7442, 6593, 2831, 7201, 1997, 1057...
34626     [101, 1045, 2572, 2047, 2000, 2023, 2609, 1998...
                                ...                        
145508    [101, 2092, 2061, 2521, 2045, 2003, 2053, 2107...
31323     [101, 11396, 2017, 4550, 2039, 2044, 2033, 204...
85790     [101, 3160, 2009, 3849, 2008, 12997, 2038, 204...
54711     [101, 5862, 16012, 7583, 2153, 2106, 2017, 319...
67881     [101, 1045, 2318, 1037, 6594, 11689, 7632, 428...
Name: text, Length: 95575, dtype: object

In [28]:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

In [29]:
padded_train = np.array([i + [0]*(max_len - len(i)) for i in tokenized[x_train.index].values])
padded_valid = np.array([i + [0]*(max_len - len(i)) for i in tokenized[x_valid.index].values])
padded_test = np.array([i + [0]*(max_len - len(i)) for i in tokenized[x_test.index].values])

attention_mask_train = np.where(padded_train != 0, 1, 0)
attention_mask_valid = np.where(padded_valid != 0, 1, 0)
attention_mask_test = np.where(padded_test != 0, 1, 0)


In [30]:
#перенесем все на GPU
masks = [attention_mask_train, attention_mask_valid, attention_mask_test]
padded_list = [padded_train, padded_valid, padded_test]

padded_cuda = []
masks_cuda = []

for padded in padded_list:
  padded_cuda.append(torch.LongTensor(padded).to(DEVICE))

for am in masks:
  masks_cuda.append(torch.LongTensor(am).to(DEVICE))

model.to(DEVICE)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [31]:
batch_size = 100
emdeddings_train = []
emdeddings_valid = []

padded_train = padded_cuda[0]
padded__valid = padded_cuda[1]
attention_mask_train = masks_cuda[0]
attention_mask_valid = masks_cuda[1]

#эмбединги на трейна 
for i in notebook.tqdm(range((padded_train.shape[0] // batch_size)+1)):
    batch = padded_train[batch_size*i:batch_size*(i+1)]
    attention_mask_batch = attention_mask_train[batch_size*i:batch_size*(i+1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    emdeddings_train.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/956 [00:00<?, ?it/s]

: 

: 

In [ ]:
#эмбединги на трейна 
for i in notebook.tqdm(range((padded_cuda[1].shape[0] // batch_size)+1)):
    batch = padded_cuda[1][batch_size*i:batch_size*(i+1)]
    attention_mask_batch = masks_cuda[1][batch_size*i:batch_size*(i+1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    emdeddings_valid.append(batch_embeddings[0][:,0,:].cpu().numpy())


  0%|          | 0/319 [00:00<?, ?it/s]

In [ ]:
features_train = np.concatenate(emdeddings_train)
features_valid = np.concatenate(emdeddings_valid)

In [ ]:
emdeddings_valid

[array([[-0.30276462, -0.06260435, -0.53611094, ..., -0.04500606,
          0.6141924 ,  0.22037649],
        [ 0.20820111,  0.0015648 , -0.17076921, ...,  0.03489113,
          0.42323813,  0.23458768],
        [-0.06729886,  0.17456762, -0.22793862, ..., -0.08660852,
          0.56272566,  0.25423023],
        ...,
        [-0.04147266,  0.07403929, -0.03090423, ...,  0.00747263,
          0.31022176,  0.3262222 ],
        [ 0.02246684,  0.0041865 , -0.16212185, ..., -0.09176595,
          0.22850019,  0.19289827],
        [-0.00807293,  0.14485742, -0.26280302, ..., -0.13031198,
          0.4486032 ,  0.26619843]], dtype=float32),
 array([[ 0.14152026,  0.08948081, -0.1778547 , ..., -0.01991412,
          0.25018588,  0.5272966 ],
        [-0.16722304, -0.05383487,  0.07556832, ..., -0.25685292,
          0.30658326,  0.47871813],
        [-0.18369345, -0.1210535 ,  0.07411571, ..., -0.11875363,
          0.1809162 ,  0.50752115],
        ...,
        [ 0.03100402,  0.18970358,  0.0

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что разобрался с векторизацией с помощью БЕРТа!
</div>

## classification 

## LR 

In [ ]:
lr = LogisticRegression(class_weight = 'balanced', random_state = 42)
lr.fit(features_train, y_train)
pred = lr.predict(features_valid)
print('f1 score on validation', f1_score(pred, y_valid)) 

f1 score on validation 0.6610926583438322


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


результат на эмбедингах не очень удовлетворительный

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Проводить кросс-валидацию на upsampled/downsampled данных – некорректно, так как баланс классов нарушен. Напомню, что внутри кросс-валидации происходит разбиение переданной в нее выборки на треин и валидацию. 
    
В случае upsampling получается так, что в треин и в валидацию (внутри кросс-валидации) вообще попадают одни и те же объекты. 

* Можно попробовать реализовать подход из статьи: https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html .
* Можно использовать параметр class_weights.
* Можно отдельно выделить валидационную выборку и изменить баланс только на треине.
* Можно вообще ничего не делать, то есть не бороться с дисбалансом. 
* Каждый их способов выше может как улучшить, так и ухудшить качество. Это поле для экспериментов.
</div>

## ctb

In [ ]:
%%time 

X_train_ctb = features_train.copy()

y_train_ctb = y_train.copy()

#без перебора гиперпаратметров, чтобы просто сэкономить ресурсы/время 
params = {'random_seed': 42,
          'eval_metric': 'F1',
          #'task_type': 'GPU', 
          'verbose':False}

model = CatBoostClassifier(**params)

model.fit(X_train_ctb, y_train_ctb)

CPU times: user 34 s, sys: 12.8 s, total: 46.9 s
Wall time: 27.9 s


In [ ]:
pred = model.predict(features_valid)
print('f1 score on validation', f1_score(pred, y_valid)) 

f1 score on validation 0.7039717563989408


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Сравнение моделей нужно провести только на кросс-валидации (с одним и тем же параметром "cv") или только на валидационной выборке.
</div>

<div class="alert alert-block alert-info">

<b>Совет: </b> Можно было попробовать специализированный БЕРТ: https://huggingface.co/unitary/toxic-bert
</div>

у кэтбуста результат чуть получше, но все еще хуже чем результат "из коробки" на tfidf

In [ ]:
from detoxify import Detoxify
def tox_coef(text):
  return Detoxify('original').predict('fucking faggot')['toxicity']

x_train['tox'] = x_train['text'].apply(lambda x: tox_coef(x))

from matplotlib import pyplot as plt 
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train, x_train['tox'])
res = pd.DataFrame({'fpr' : fpr, 'tpr':tpr,'thresholds':thresholds })
plt.title('roc auc curve')
sns.lineplot(x=res['fpr'], y = res['tpr'])
plt.show()

#найдем трешхолд, где tpr максимальный, а fpr минимальный
res['res'] = res.fpr - res.tpr
res.sort_values(by ='res')

In [ ]:
from matplotlib import pyplot as plt 
from sklearn.metrics import roc_curve
import seaborn as sns 

fpr, tpr, thresholds = roc_curve(y_train,  model.predict_proba(features_train)[:,1])
res = pd.DataFrame({'fpr' : fpr, 'tpr':tpr,'thresholds':thresholds })
plt.title('roc auc curve')
sns.lineplot(x=res['fpr'], y = res['tpr'])
plt.show()

#найдем трешхолд, где tpr максимальный, а fpr минимальный
res['res'] = res.fpr - res.tpr
res.sort_values(by ='res')
threshold = res.sort_values(by ='res')['thresholds'].values[0]
valid_pred = model.predict_proba(features_valid)[:,1]

predictions_after_threshold = (valid_pred>=threshold)*1
print('f1 score on validation', f1_score(predictions_after_threshold, y_valid)) 

## best model test

In [ ]:
lr.fit(x_train, y_train) 
pred = l
r.predict(x_test)
print(f1_score(pred, y_test))

#Итог

Лучше себя показала модель линейной регресси: быстро, просто, дешево. 

F1 мера около 0.82, что не так уж плохо. 

Что еще можно сделать: 
  - поиграться с трешхолдом 
  - апсемплинг 
  - поперебирать еще параметры катбуста 
  - добавить другой бустинг 
  - посмотреть другие модели для эмбедингов

<div class="alert alert-block alert-success">
<b>Успех:</b> Приятно видеть вывод в конце проекта!
</div>